# Data engineering homework

##  1. Dimensions vs metrics.


Have a look at all fields in all reports that we deliver to clients https://improvado.io/docs-section-topic/data-dictionary-available-report-types

There are two kind of fields:
— dimensions (country, banner_id, campaign_name ..)
— metrics: (clicks, spends, impressions …)

We need an estimation for a question: what part of the fields are metrics? It is not always easy to say exactly if this field metric is neither dimension. Ok to have answe liker: from 25% to 35% (round answer to integer).

In [288]:
import numpy as np
import pandas as pd

In [289]:
import re

In [290]:
df = pd.read_csv('data/Data_Dictionary_for_CSV_2022_09_29.csv') #input data

In [291]:
df.shape

(30655, 5)

### Search for duplicates.

Complete duplicates of the string

In [292]:
df_duplicates = df[df.duplicated()]
print(f'Duplicates: {df_duplicates.shape[0]}')

Duplicates: 3


Removing duplicates

In [293]:
df = df.drop_duplicates()
df_copy = df.copy()

In [294]:
df.shape[0]

30652

In [295]:
df.dtypes

data_source_title    object
report_type_title    object
field_title          object
sql_field_name       object
api_field_name       object
dtype: object

Checking for zeros

In [296]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30652 entries, 0 to 30654
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   data_source_title  30652 non-null  object
 1   report_type_title  30652 non-null  object
 2   field_title        30652 non-null  object
 3   sql_field_name     30652 non-null  object
 4   api_field_name     30652 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [297]:
# converting all columns to string type
df = df.astype('string')
print(df.dtypes)

data_source_title    string
report_type_title    string
field_title          string
sql_field_name       string
api_field_name       string
dtype: object


A brief overview of the information from the table:

In [298]:
df.head()

,data_source_title,report_type_title,field_title,sql_field_name,api_field_name
0,ACME,Transactions,Event Id,event_id,EventId
1,ACME,Transactions,Discount Code,discount_code,DiscountCode
2,ACME,Transactions,Customer Id,customer_id,CustomerId
3,ACME,Transactions,Transaction Amount,transaction_amount,TransactionAmount
4,ACME,Transactions,Quantity,quantity,Quantity


In [299]:
df.describe(include = 'all')

,data_source_title,report_type_title,field_title,sql_field_name,api_field_name
count,30652,30652,30652,30652,30652
unique,174,938,9158,8620,9256
top,Facebook,All Fields,Clicks,campaign_id,???
freq,5171,1084,418,594,809


In [300]:
df.value_counts().nlargest(10)

data_source_title  report_type_title  field_title             sql_field_name          api_field_name        
ACME               Transactions       Add On                  add_on                  AddOn                     1
Iterable           Campaigns          Campaign ID             campaign_id             id                        1
                                      Total Emails Delivered  total_emails_delivered  Total Emails Delivered    1
                                      Total Emails Clicked    total_emails_clicked    Total Emails Clicked      1
                                      Total Emails Bounced    total_emails_bounced    Total Emails Bounced      1
                                      Total Email Sends       total_email_sends       Total Email Sends         1
                                      Total Email Send Skips  total_email_send_skips  Total Email Send Skips    1
                                      Total Email Opens       total_email_opens       Total E

### Creating a new feature:

In [301]:
df['common_title'] = df['field_title'].str.lower() + ' '\
                   + df['sql_field_name'].str.lower() + ' '\
                   + df['api_field_name'].str.lower()
df['common_title'] = df['common_title'].astype('string')  # common titles info
df.head()

,data_source_title,report_type_title,field_title,sql_field_name,api_field_name,common_title
0,ACME,Transactions,Event Id,event_id,EventId,event id event_id eventid
1,ACME,Transactions,Discount Code,discount_code,DiscountCode,discount code discount_code discountcode
2,ACME,Transactions,Customer Id,customer_id,CustomerId,customer id customer_id customerid
3,ACME,Transactions,Transaction Amount,transaction_amount,TransactionAmount,transaction amount transaction_amount transact...
4,ACME,Transactions,Quantity,quantity,Quantity,quantity quantity quantity


The function of removing duplicate words in a string:

In [302]:
def get_unique_words(str1):
    words = str1.split()
    return " ".join(sorted(set(words), key=words.index))

In [303]:
df['common_title'] = df['common_title'].map(get_unique_words)
df.head()

,data_source_title,report_type_title,field_title,sql_field_name,api_field_name,common_title
0,ACME,Transactions,Event Id,event_id,EventId,event id event_id eventid
1,ACME,Transactions,Discount Code,discount_code,DiscountCode,discount code discount_code discountcode
2,ACME,Transactions,Customer Id,customer_id,CustomerId,customer id customer_id customerid
3,ACME,Transactions,Transaction Amount,transaction_amount,TransactionAmount,transaction amount transaction_amount transact...
4,ACME,Transactions,Quantity,quantity,Quantity,quantity


### Search for duplicates in the column: 'common_title'.

In [304]:
title_duplicates = df[df.duplicated(subset=['common_title'])]
print(f'Duplicates in title: {title_duplicates.shape[0]}')

Duplicates in title: 19541


Removing duplicates in the column.

In [305]:
df_dedupped = df.drop_duplicates(subset=['common_title'], ignore_index=True)
df_dedupped.shape

(11111, 6)

In [306]:
display(df_dedupped['common_title'].nunique())

11111

### Creating masks from words and parts of words, selected manually by meaning.

Maybe an alternative way to parse information from: 
<a href="https://www.googleapis.com/analytics/v3/metadata/ga/columns">to google api columns</a>
or other sources.

```python
import json
import urllib.request

with urllib.request.urlopen("https://www.googleapis.com/analytics/v3/metadata/ga/columns") as url:
    data = json.load(url)
    # print(data['items'][0])
    dict_val0 = data['items'][0]
    print(dict_val0['attributes']['type'])
```

Sets of words for masks:

In [307]:
must_contain_exact = [
    'id', 'name', 'title', 'code', 'campaign', 'type', 'status', 'date',
    'method', 'time', 'city', 'region', 'country', 'on', 'channel', 'type', 'form', 'post',
    'click', 'color', 'map', 'dimension', 'link', 'cookies', 'signature', 'login', 'auth', 
    'blogs', 'report', 'is', 'ticket', 'chat', 'media', 'email', 'phone', 'address', 'url',
    'source', 'by', 'agent', 'shared', 'alias', 'subject', 'recipient', 'sender', 'language',
    'category', 'moderator', 'version', 'start', 'satrted', 'complete' 'completed', 'has',
    'browser', 'os', 'action', 'data', 'state', 'tag', 'domain', 'last', 'first',' size',
    'description', 'image', 'content', 'definition', 'private', 'public', 'weight', 'banner',
    'impressions', 'sent', 'text', 'new', 'completions',
]
must_contain_subs = [
    'id', 'name', 'title', 'code', 'campaign', 'type', 'status', 'date',
    'method', 'time', 'hour', 'day', 'week', 'month', 'city',
    'region', 'countr', 'channel', 'type', 'form', 'post',
    'click', 'color', 'map', 'dimension', 'link', 'cookie',
    'group', 'signature', 'login', 'auth', '_at', 'by_', 'blogs',
    'report', 'email', 'phone', 'address', 'url', 'source', 'category',
    'version', 'os', 'action', 'data', 'ed', 'tag', 'details', 'completions',
    'size', 'weight',
]

not_contain = [
    'value', 'total', 'amount', 'quantity', 'balance', 'clicks', 'price', 
    'views', 'occurrences', 'average', 'rate', 'cost', 'metrics', 
    'score', 'rating', 'revenue', 'accuracy', 'number', '%', 'frequency',
    'income', 'statistics', 'percentage',
]
not_contain_subs = [
    'value', 'total', 'amount', 'quantity', 'balance', 'clicks', 'price', 
     'occurrences', 'average', 'rate', 'cost', 'metrics', 
     'score', 'rating', 'avg', 'mean', 'count', 'revenue',
     'accuracy', 'number', 'freq', 'statistics'
]

Applying these masks leaves lines related to dimensions:

In [308]:
p = r'\b(?:{})\b'.format('|'.join(map(re.escape, must_contain_exact)))
pat_include = re.compile(p)

p = r'\b(?:{})\b'.format('|'.join(map(re.escape, not_contain)))
pat_not_include = re.compile(p)

in_list_exact = [bool(pat_include.search(x)) for x in df_dedupped['common_title']]
in_list_subs = df_dedupped['common_title'].str.contains('|'.join(must_contain_subs)).tolist()

not_in_list = [bool(pat_not_include.search(x)) for x in df_dedupped['common_title']]
not_in_subs = df_dedupped['common_title'].str.contains('|'.join(not_contain_subs)).tolist()


mask = np.logical_or(in_list_exact, np.logical_and(in_list_subs, np.logical_not(np.logical_or(not_in_list, not_in_subs))))
                   
                   
display(df_dedupped[mask].count()[0])

7526

**To check the result:**

In [309]:
# df_dedupped[np.logical_not(mask)]['common_title'].head(50)
df_dedupped[np.logical_not(mask)]['common_title'].tail(50)
# df_dedupped[np.logical_not(mask)]['common_title'].count()

10894                            impression ad_impressions
10897      youtube ad revenue estimated_youtube_ad_revenue
10898    partner revenue reserved estimated_partner_ad_...
10899    partner revenue auction estimated_partner_ad_a...
10900      partner ad revenue estimated_partner_ad_revenue
10901            partner revenue estimated_partner_revenue
10911                               like snippet.likecount
10912                    reply reply_count totalreplycount
10914     viewer rating viewer_rating snippet.viewerrating
10923                           views statistics.viewcount
10924    videos published videos_published statistics.v...
10925               subscribers statistics.subscribercount
10926                     comments statistics.commentcount
10930    average view percentage average_view_percentag...
10931    average view duration average_view_duration av...
10933    comment count statistics_comment_count statist...
10934    favorite count statistics_favorite_count stati.

### Result:

In [310]:
dims = round(df_dedupped[mask].count()[0] / df_dedupped.shape[0] * 100.0)
metrics = 100 - dims
print(metrics)

32


### In total, according to my calculations, metrics account for 32 percent of all characteristics.

Made by Skosyrskiy Anton. I will be glad to review.  
My email: ant.sckosyrsckij@ynadex.ru.